In [2]:
import threading
import time

# Update in transaction_TX function
def transaction_TX(amount, operation):
    """
    Transaction TX: Performs the user-specified operation (add or deduct) and writes back after delay.
    """
    print("\n[TX] Starts")
    time.sleep(1)  # TX starts at t1
    local_balance = database["account_A"]
    print(f"[TX - t1] Reads balance: {local_balance}")

    time.sleep(1)  # TX operates at t2
    if operation == "deduct":
        local_balance -= amount
        print(f"[TX - t2] Deducts {amount}, new local balance: {local_balance}")
    elif operation == "add":
        local_balance += amount
        print(f"[TX - t2] Adds {amount}, new local balance: {local_balance}")

    time.sleep(3)  # TX writes back at t6
    database["account_A"] = local_balance
    print(f"[TX - t6] Writes back balance: {database['account_A']}")  # Ensure t6 prints before t7

# Update in transaction_TY function
def transaction_TY(amount, operation):
    """
    Transaction TY: Performs the user-specified operation (add or deduct) and writes back after delay.
    """
    time.sleep(2)  # TY starts at t3
    print("\n[TY] Starts")
    local_balance = database["account_A"]
    print(f"[TY - t3] Reads balance: {local_balance}")

    time.sleep(1)  # TY operates at t4
    if operation == "deduct":
        local_balance -= amount
        print(f"[TY - t4] Deducts {amount}, new local balance: {local_balance}")
    elif operation == "add":
        local_balance += amount
        print(f"[TY - t4] Adds {amount}, new local balance: {local_balance}")

    time.sleep(2)  # TY writes back at t7
    print(f"[TY - t7] Writes back balance: {local_balance}")  # Ensures t7 prints after t6
    database["account_A"] = local_balance


# User inputs
print("=== Lost Update Problem Simulation ===")
initial_balance = int(input("Enter the initial balance of account A: "))

# Input for TX
print("\n[TX Configuration]")
tx_operation = input("Enter the operation for TX (add/deduct): ").strip().lower()
tx_amount = int(input(f"Enter the amount to {tx_operation} by TX: "))

# Input for TY
print("\n[TY Configuration]")
ty_operation = input("Enter the operation for TY (add/deduct): ").strip().lower()
ty_amount = int(input(f"Enter the amount to {ty_operation} by TY: "))

# Shared database resource
database = {"account_A": initial_balance}

# Threads for concurrent transactions
tx_thread = threading.Thread(target=transaction_TX, args=(tx_amount, tx_operation))
ty_thread = threading.Thread(target=transaction_TY, args=(ty_amount, ty_operation))

# Start transactions
tx_thread.start()
ty_thread.start()

# Wait for both to finish
tx_thread.join()
ty_thread.join()

# Final balance
print(f"\nFinal balance after simulation: {database['account_A']}")

# Explanation of Lost Update Problem
print("\n=== Explanation of Lost Update Problem ===")
if tx_operation == "add":
    tx_effect = f"adds {tx_amount}"
elif tx_operation == "deduct":
    tx_effect = f"deducts {tx_amount}"

if ty_operation == "add":
    ty_effect = f"adds {ty_amount}"
elif ty_operation == "deduct":
    ty_effect = f"deducts {ty_amount}"

print(
    f"The issue arises because TX {tx_effect} and TY {ty_effect} concurrently, but they "
    f"operate on outdated balances due to delays in writing back to the database.\n"
    f"At t6, TX wrote the balance as {initial_balance + tx_amount if tx_operation == 'add' else initial_balance - tx_amount}.\n"
    f"However, at t7, TY overwrote the balance as {initial_balance + ty_amount if ty_operation == 'add' else initial_balance - ty_amount}, "
    f"resulting in TX's update being lost.\n"
    f"This demonstrates the Lost Update Problem: updates made by one transaction can be overwritten by another transaction "
    f"when proper concurrency control mechanisms are not in place."
)


=== Lost Update Problem Simulation ===
Enter the initial balance of account A: 300

[TX Configuration]
Enter the operation for TX (add/deduct): add
Enter the amount to add by TX: 50

[TY Configuration]
Enter the operation for TY (add/deduct): deduct
Enter the amount to deduct by TY: 100

[TX] Starts
[TX - t1] Reads balance: 300
[TX - t2] Adds 50, new local balance: 350

[TY] Starts
[TY - t3] Reads balance: 300
[TY - t4] Deducts 100, new local balance: 200
[TX - t6] Writes back balance: 350[TY - t7] Writes back balance: 200


Final balance after simulation: 200

=== Explanation of Lost Update Problem ===
The issue arises because TX adds 50 and TY deducts 100 concurrently, but they operate on outdated balances due to delays in writing back to the database.
At t6, TX wrote the balance as 350.
However, at t7, TY overwrote the balance as 200, resulting in TX's update being lost.
This demonstrates the Lost Update Problem: updates made by one transaction can be overwritten by another transact